## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [2]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number
from math import cos, sin, exp, log

@dataclass
class Dual:
    real: float
    dual: float = 0.0

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual():
                return Dual(self.real + other.real, self.dual + other.dual)
            case Number():
                return Dual(float(other) + self.real, self.dual)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual():
                return Dual(self.real * other.real, self.real * other.dual + self.dual * other.real)
            case Number():
                return Dual(float(other) * self.real, float(other) * self.dual)  
    
    def __neg__(self) -> "Dual":
        return Dual(-self.real, -self.dual)
    
    def __pos__(self) -> "Dual":
        return Dual(+self.real, +self.dual)
    
    def __abs__(self) -> "Dual":
        return Dual(abs(self.real), abs(self.dual))

    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual():
                return Dual(self.real - other.real, self.dual - other.dual)
            case Number():
                return Dual(float(other) - self.real, self.dual)
    
    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        if other.real == 0:
            raise ZeroDivisionError("Exception Zero Division!")
        
        if isinstance(other, Dual):
            other = other
        else: 
            other = Dual(real=other)
        
        real = self.real / other.real
        dual = (self.dual * other.real - self.real * other.dual)/ (other.real * other.real)
        
        return Dual(real, dual)
    
    def __rtruediv__(self, other: float) -> "Dual":
        return Dual(real=other) / self

    def __pow__(self, power: Union["Dual", Number]) -> "Dual": 
        if self.real <= 0:
            raise ZeroDivisionError("Float division by zero")

        if isinstance(power, Dual):
            other = power
            real = self.real ** other.real
            dual = real * ((self.dual / self.real) * other.real + log(self.real) * other.dual)
            return Dual(real, dual) 
        
        real = self.real ** power
        dual = real * (self.dual / self.real) * power
        return Dual(real, dual)
    
    def __rpow__(self, other: Union["Dual", Number]) -> "Dual":
        if other <= 0:
            raise AssertionError(f"Base must be nonnegative but is {other}")

        return Dual(other) ** self

    __radd__ = __add__
    __rmul__ = __mul__  
    __rsub__ = __sub__
    

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).dual

In [6]:
import pytest

#Реализуем проверку каждого отдельного метода с использованием assert
# Проверка унарных операций __neg__, __pos__, __abs__. Без реализации этих методов в классе будет возникать TypeError: bad operand type for unary (+, -, abs()): 'Dual'
# __neg__:
f = lambda x: -x 
f_diff = diff(f)
assert f_diff(2) == -1, "__neg__ is not right"

# __pos__:
f = lambda x: +x 
f_diff = diff(f)
assert f_diff(2) == 1, "__pos__ is not right"

# __abs__:
f = lambda x: abs(-x)
f_diff = diff(f)
assert f_diff(2) == 1, "__abs__ is not right"

#Проверим поддержку операции вычитания __sub__:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x - 2 * x - 2

f_diff = diff(f)
assert f_diff(2) == 18, "__sub__ is not right"

#Проверим поддержку операции деления __truediv__:
#1. Проверка деления на ноль
def f(x: float) -> float:
    return x / 0

f_diff = diff(f)

def test_zero_division():
    with pytest.raises(ZeroDivisionError):
        f_diff(2)

#2.
def f(x: float) -> float:
    return 5 / (x * x) - x * x / 2 - 2

f_diff = diff(f)
assert f_diff(2) == -3.25, "__truediv__ is not right"

#Проверим операцию возведения в степень __pow__:
#1
f = lambda x: x ** 4 
f_diff = diff(f)
assert f_diff(2) == 32, "__pow__ is not right"

#2
f = lambda x: 4 ** x
f_diff = diff(f)
assert round(f_diff(2), 4) == 22.1807, "__pow__ is not right"

#3
f = lambda x: x ** x
f_diff = diff(f)
assert round(f_diff(2), 5) == 6.77259, "__pow__ is not right"

#4
f = lambda x: 0 ** x
f_diff = diff(f)
def test_negative_base():
    with pytest.raises(AssertionError):
        f_diff(2)

#5
f = lambda x: x ** 5
f_diff = diff(f)
def test_negative_base():
    with pytest.raises(ZeroDivisionError):
        f_diff(0)

#Проверим компиляцию всех операций:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x / (x ** 4 + 2) - 2 * x - 2

f_diff = diff(f)
f_diff(2)
assert round(f_diff(2),4) == -2.8642, "wrong answer"

## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [83]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number
from math import cos, sin, exp, log

@dataclass
class Dual:
    real: float
    dual: float = 0.0

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual():
                return Dual(self.real + other.real, self.dual + other.dual)
            case Number():
                return Dual(float(other) + self.real, self.dual)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual():
                return Dual(self.real * other.real, self.real * other.dual + self.dual * other.real)
            case Number():
                return Dual(float(other) * self.real, float(other) * self.dual)  
    
    def __neg__(self) -> "Dual":
        return Dual(-self.real, -self.dual)
    
    def __pos__(self) -> "Dual":
        return Dual(+self.real, +self.dual)
    
    def __abs__(self) -> "Dual":
        return Dual(abs(self.real), abs(self.dual))

    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual():
                return Dual(self.real - other.real, self.dual - other.dual)
            case Number():
                return Dual(float(other) - self.real, self.dual)
    
    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        if other.real == 0:
            raise ZeroDivisionError("Exception Zero Division!")
        
        if isinstance(other, Dual):
            other = other
        else: 
            other = Dual(real=other)
        
        real = self.real / other.real
        dual = (self.dual * other.real - self.real * other.dual)/ (other.real * other.real)
        
        return Dual(real, dual)
    
    def __rtruediv__(self, other: float) -> "Dual":
        return Dual(real=other) / self

    def __pow__(self, power: Union["Dual", Number]) -> "Dual": 
        if self.real <= 0:
            raise ZeroDivisionError("Float division by zero")

        if isinstance(power, Dual):
            other = power
            real = self.real ** other.real
            dual = real * ((self.dual / self.real) * other.real + log(self.real) * other.dual)
            return Dual(real, dual) 
        
        real = self.real ** power
        dual = real * (self.dual / self.real) * power
        return Dual(real, dual)
    
    def __rpow__(self, other: Union["Dual", Number]) -> "Dual":
        if other <= 0:
            raise AssertionError(f"Base must be nonnegative but is {other}")

        return Dual(other) ** self

    __radd__ = __add__
    __rmul__ = __mul__  
    __rsub__ = __sub__

    def sin(self) -> "Dual":
        real = sin(self.real)
        dual = cos(self.real) * self.dual
        return Dual(real, dual)
    
    def cos(self) -> "Dual":
        real = cos(self.real)
        dual = -sin(self.real) * self.dual
        return Dual(real, dual)

    def exp(self) -> "Dual":
        real = exp(self.real)
        dual = real * self.dual
        return Dual(real, dual)
    
    def log(self) -> "Dual":
        if self.real == 0:
            raise AssertionError(f"log(0) is not defined")

        real = log(self.real)
        dual = self.dual / self.real
        return Dual(real, dual)
    

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).dual

In [77]:
from math import pi, exp

#Реализуем проверку каждого отдельного метода с использованием assert
def test_sin():
    f = lambda x: (x ** 2).sin()
    f_diff = diff(f)
    assert f_diff(pi) == 2 * pi * cos(pi**2), "sin() is not right"

def test_cos():
    f = lambda x: (x ** 2).cos()
    f_diff = diff(f)
    assert f_diff(pi) == -2 * pi * sin(pi**2), "cos() is not right"

def test_exp():
    f = lambda x: (x ** 2).exp()
    f_diff = diff(f)
    assert f_diff(pi) == 2 * pi * exp(pow(pi, 2)), "exp() is not right"

def test_log():
    f = lambda x: (x ** 2).log()
    f_diff = diff(f)
    assert f_diff(pi) == 2 / pi, "log() is not right"

    with pytest.raises(AssertionError):
        f_diff(0)


## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [113]:
from scipy.misc import derivative

def f(x: float) -> float:
    return (x**3) - 2*(x**2)*(x**2 + 2) + (x**3 + x + 5)

print(diff(f)(2))
print(derivative(f, 2.))
print('Right answer is -55')

-55.0
-69.0
Right answer is -55


In [93]:
def f(x: float) -> float:
    return 81*x / (x+(x**2))

print(diff(f)(2))
print(derivative(f, 2.))

print('Right answer is -9')

-9.0
-10.125
Right answer is -9


In [97]:
def f(x: float) -> float:
    return sin(x)/(cos(x)+(x**2))

print(derivative(f, 2.))

def f(x: float) -> float:
    return x.sin()/(x.cos()+(x**2))

print(diff(f)(2))

print('Right answer is -0.334925')

-0.2643422639065857
-0.3349249363278422
Right answer is -0.334925
